In [1]:
import gc
import torch
import os

# Set device
torch_device = "cuda" if torch.cuda.is_available() else "mps" if torch.backends.mps.is_available() else "cpu"
if "mps" == torch_device: os.environ['PYTORCH_ENABLE_MPS_FALLBACK'] = "1"

def clear_cache():
    gc.collect()
    if torch_device == 'cuda':
        torch.cuda.empty_cache()
    elif torch_device == 'mps':
        torch.mps.empty_cache()

# 1. Indexing

In [2]:
from langchain_community.document_loaders import PyPDFLoader

loader = PyPDFLoader("./sample_docs/Surya_Resume.pdf")
pages = loader.load_and_split()


#from langchain_community.document_loaders import PDFMinerLoader
#loader = PDFMinerLoader("./sample_docs/Surya_Resume.pdf")
#data = loader.load()


In [3]:
print('Num pages: ', len(pages))
print(pages[0].page_content)

Num pages:  2
SURYA KUMAR SARIPELLA H 27 Richardson Rd, Burlington, MA 01803 https://www.linkedin.com/in/surya-kumar-saripella/ È (669) 246.0372 : s.suryakumar@gmail.com    SUMMARY   Accomplished leader with proven skills in driving technology strategy and implementation to fulfill business objectives. Expert in fostering an environment rooted in data-centric decision-making and adept at steering cross-functional teams within the Financial and Technology industries. Skilled in:  ü Data and AI Strategy & Implementation ü Strategic Partnerships & Stakeholder Relations  ü Tech Strategy / Product Management ü AI/ML Models & Analytics ü Business Integration & Growth ü Scaling Teams & Nurturing Talent   PROFESSIONAL EXPERIENCE   DISTILL.IO                          BURLINGTON, MA Developing data pipelines to track unstructured website changes for enterprise notifications CHIEF INNOVATION OFFICER / HEAD OF AI STRATEGY MAY 2021 – PRESENT   ü Formulated the AI strategy and roadmap to enhance not

# 2. Indexing: Split into documents

In [4]:
from langchain_text_splitters import RecursiveCharacterTextSplitter

text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=500, chunk_overlap=100, add_start_index=True
)
all_splits = text_splitter.split_documents(pages)
len(all_splits)

#from langchain_text_splitters import CharacterTextSplitter
#char_text_splitter = CharacterTextSplitter(
#    separator='\n\n',
#    chunk_size=1000,
#    chunk_overlap=200,
#    length_function=len,
#    is_separator_regex=False,
#)
#texts = char_text_splitter.create_documents(pages[0].page_content)
#print(len(texts))


15

In [5]:
print(all_splits[10])

page_content='SURYA KUMAR SARIPELLA H 27 Richardson Rd, Burlington, MA 01803 https://www.linkedin.com/in/surya-kumar-saripella/ È (669) 246.0372 : s.suryakumar@gmail.com     MORGAN STANLEY            MUMBAI, INDIA Global Equity Derivatives Strategy SENIOR ASSOCIATE APR 2009 – JAN 2012   ü Developed a Cross-Asset volatility monitor to analyze volatility metrics (volatility, term structure, skew etc.) across Equities, FX, Interest Rates, Commodities & Credit to identify market dislocations ü Designed' metadata={'source': './sample_docs/Surya_Resume.pdf', 'page': 1, 'start_index': 0}


In [6]:
print(all_splits[0].page_content)

SURYA KUMAR SARIPELLA H 27 Richardson Rd, Burlington, MA 01803 https://www.linkedin.com/in/surya-kumar-saripella/ È (669) 246.0372 : s.suryakumar@gmail.com    SUMMARY   Accomplished leader with proven skills in driving technology strategy and implementation to fulfill business objectives. Expert in fostering an environment rooted in data-centric decision-making and adept at steering cross-functional teams within the Financial and Technology industries. Skilled in:  ü Data and AI Strategy &


# Step 3: Indexing: Store

In [7]:
from langchain_community.embeddings import HuggingFaceEmbeddings
from sentence_transformers import SentenceTransformer
from langchain_community.vectorstores import Chroma

#model = SentenceTransformer('sentence-transformers/all-MiniLM-L6-v2')

#Embedding Model options (dimensions must be 768)

#model_name = 'sentence-transformers/all-mpnet-base-v2' # Embedding size is 768
#model_name='sentence-transformers/all-MiniLM-L6-v2' # Embedding size is 328 (won't work)
model_name = 'sentence-transformers/all-distilroberta-v1' # Embedding size is 768


model_kwargs = {'device':'mps'} # allows using GPU for embeddings
encode_kwargs = {'normalize_embeddings': True} # normalizes embedding vector (between 0 to 1)
embeddings = HuggingFaceEmbeddings(
    model_name=model_name,
    model_kwargs=model_kwargs,
    encode_kwargs=encode_kwargs
)

vectorstore = Chroma.from_documents(documents=all_splits, embedding=embeddings)

/Users/surya/Documents/personal/kaggle/venv_kaggle/lib/python3.11/site-packages/torch/_utils.py:831: UserWarning: TypedStorage is deprecated. It will be removed in the future and UntypedStorage will be the only storage class. This should only matter to you if you are using storages directly.  To access UntypedStorage directly, use tensor.untyped_storage() instead of tensor.storage()
  return self.fget.__get__(instance, owner)()


# Step 4: Retreival & Generation: Retrieve

In [8]:
#retriever = vectorstore.as_retriever(search_type="mmr", search_kwargs={"k": 8})
retriever = vectorstore.as_retriever(search_type="similarity", search_kwargs={"k": 8})

# Step 5: Retreival & Generation: Generate

In [9]:
from langchain_community.chat_models import ChatOllama

#Defining the models to be used

llm_ollama = ChatOllama(model='llama2')
#llm_ollama = ChatOllama(model='bakllava')
#llm_ollama = ChatOllama(model='mixtral')
#llm_ollama = ChatOllama(model='mistral')
#llm_ollama = ChatOllama(model='gemma')


In [10]:
from langchain import hub
prompt = hub.pull("rlm/rag-prompt")

In [11]:
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnablePassthrough


def format_docs(docs):
    return "\n\n".join(doc.page_content for doc in docs)


rag_chain = (
    {"context": retriever | format_docs, "question": RunnablePassthrough()}
    | prompt
    | llm_ollama
    | StrOutputParser()
)

In [12]:
q1 = "Which all companies did Surya work for?"
q2 = "Which all cities did Surya work so far?"
q3 = "What are Surya's academic qualifications?"
q4 = "Where does Surya currently reside?"

#Contact Info  (does not seem to work)
q5 = "Give me Surya's Phone Number"
q6 = "Give me Surya's e-mail address"
q7 = "Can you extract Surya's contact information which is at the top of the document?"

q8 = "What job titles should Surya look for?"

#


In [13]:
question = q2

for chunk in rag_chain.stream(question):
    print(chunk, end="", flush=True)

Surya Kumar Saripella has worked in the following cities:

* Mumbai, India (for work at Quantum Phinance Consulting Pvt Ltd and Morgan Stanley)
* Burlington, MA (for work at Distill.io and as a Chief Innovation Officer/Head of AI)

Surya Kumar Saripella's professional experience includes working as a Senior Associate at Morgan Stanley in Mumbai, India, where he developed a cross-asset volatility monitor to analyze volatility metrics across various asset classes. He also worked at Distill.io in Burlington, MA, where he developed data pipelines to track unstructured website changes for enterprise notifications. Additionally, Surya Kumar Saripella has experience in strategic partnerships and stakeholder relations, tech strategy and product management, AI/ML models and analytics, business integration and growth, and scaling teams and nurturing talent.

In [14]:
prompt

ChatPromptTemplate(input_variables=['context', 'question'], messages=[HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['context', 'question'], template="You are an assistant for question-answering tasks. Use the following pieces of retrieved context to answer the question. If you don't know the answer, just say that you don't know. Use three sentences maximum and keep the answer concise.\nQuestion: {question} \nContext: {context} \nAnswer:"))])

In [16]:
docs = retriever.get_relevant_documents(q2)

In [18]:
docs[0]

Document(page_content='SURYA KUMAR SARIPELLA H 27 Richardson Rd, Burlington, MA 01803 https://www.linkedin.com/in/surya-kumar-saripella/ È (669) 246.0372 : s.suryakumar@gmail.com    SUMMARY   Accomplished leader with proven skills in driving technology strategy and implementation to fulfill business objectives. Expert in fostering an environment rooted in data-centric decision-making and adept at steering cross-functional teams within the Financial and Technology industries. Skilled in:  ü Data and AI Strategy &', metadata={'page': 0, 'source': './sample_docs/Surya_Resume.pdf', 'start_index': 0})